In [2]:
import pandas as pd
import re

# --- 설정 ---
# 네 최종 결과 CSV 파일 이름
final_dataset_filename = 'augmented_dataset_n5_20250613_divide_date_deduplicated.csv' 

try:
    df = pd.read_csv(final_dataset_filename)
    print(f"'{final_dataset_filename}' 파일 로드 성공! 총 {len(df)}개의 샘플(행)이 있습니다.\n")

    # --- 각 샘플이 긴급 주문을 포함하는지 여부 분석 ---
    
    # 'order' 컬럼에서 priority(네 번째 숫자) 값을 추출하는 정규표현식
    # 예: "• date169 item87 45500 1 196" -> 여기서 1을 찾음
    priority_pattern = re.compile(r"•\s+\S+\s+\S+\s+\d+\s+(\d+)\s+\d+")
    
    def has_urgent_order(order_string):
        """order 문자열을 파싱해서 긴급 주문(priority > 0)이 하나라도 있는지 확인합니다."""
        if not isinstance(order_string, str):
            return False
        
        priorities = priority_pattern.findall(order_string)
        for p_str in priorities:
            if int(p_str) > 0:
                return True # 긴급 주문이 하나라도 있으면 True 반환
        return False # 긴급 주문이 하나도 없으면 False 반환

    # 'has_urgent_orders' 컬럼 추가: 각 샘플이 긴급 주문을 포함하면 True
    df['has_urgent_orders'] = df['order'].apply(has_urgent_order)
    
    # 'is_score_zero' 컬럼 추가: objective 점수가 0이면 True
    df['is_score_zero'] = (df['objective'] == 0)

    # --- 교차 분석표(Crosstab)로 결과 요약 ---
    
    print("--- '긴급 주문 유무'와 '점수 0점 여부'의 교차 분석 결과 ---")
    analysis_table = pd.crosstab(df['has_urgent_orders'], df['is_score_zero'])
    print(analysis_table)
    
    
    # --- 추가 분석 ---
    print("\n--- 상세 분석 ---")
    
    # 1. 긴급 주문이 없어서 점수가 0점인 경우
    no_urgent_and_zero_score = len(df[(df['has_urgent_orders'] == False) & (df['is_score_zero'] == True)])
    print(f"1. 긴급 주문이 아예 없어서 점수가 0점인 샘플 수: {no_urgent_and_zero_score} 건")
    
    # 2. 긴급 주문이 있었음에도 불구하고, GA가 완벽하게 해결해서 0점을 받은 경우
    urgent_and_zero_score = len(df[(df['has_urgent_orders'] == True) & (df['is_score_zero'] == True)])
    print(f"2. 긴급 주문이 있었지만 '완벽하게' 해결하여 0점을 받은 샘플 수: {urgent_and_zero_score} 건")

    # 3. 긴급 주문이 있어서 페널티를 받은 경우
    urgent_and_non_zero_score = len(df[(df['has_urgent_orders'] == True) & (df['is_score_zero'] == False)])
    print(f"3. 긴급 주문이 있어서 페널티를 받은(0점 초과) 샘플 수: {urgent_and_non_zero_score} 건")
    
    # 4. 긴급 주문이 없는데도 0점이 아닌 경우 (이론상 거의 없어야 함)
    no_urgent_and_non_zero_score = len(df[(df['has_urgent_orders'] == False) & (df['is_score_zero'] == False)])
    print(f"4. 긴급 주문이 없는데도 페널티를 받은 경우(과잉생산 등): {no_urgent_and_non_zero_score} 건")

except FileNotFoundError:
    print(f"오류: '{final_dataset_filename}' 파일을 찾을 수 없습니다.")

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
'augmented_dataset_n5_20250613_divide_date_deduplicated.csv' 파일 로드 성공! 총 365개의 샘플(행)이 있습니다.

--- '긴급 주문 유무'와 '점수 0점 여부'의 교차 분석 결과 ---
is_score_zero      False  True 
has_urgent_orders              
True                 327     38

--- 상세 분석 ---
1. 긴급 주문이 아예 없어서 점수가 0점인 샘플 수: 0 건
2. 긴급 주문이 있었지만 '완벽하게' 해결하여 0점을 받은 샘플 수: 38 건
3. 긴급 주문이 있어서 페널티를 받은(0점 초과) 샘플 수: 327 건
4. 긴급 주문이 없는데도 페널티를 받은 경우(과잉생산 등): 0 건
